# Умная система фильтрации отзывов - Revi AI

Интеллектуальный микросервис для анализа отзывов, который  не только классифицирует обратную связь и определяет её тональность, но и автоматически выделяет ключевые проблемы и генерирует рекомендации для их решения

## Шаг 0. Загрузка данных и их обработка

In [ ]:
!pip install pymorphy2

In [40]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import openpyxl
import re
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
import plotly.express as px
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer
import numpy as np
from torch.utils.data import DataLoader, Dataset
import re
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pymorphy2 import MorphAnalyzer
import pandas as pd

In [4]:
df1 = pd.read_excel('Категоризация_проблем_негатив,_подробное_деление.xlsx')
df2 = pd.read_excel('Отзывы для категоризации.xlsx')
df3 = pd.read_excel('Отзывы_спам,_модерация,_проблемы,_корректные.xlsx')
df2.rename(columns={'Unnamed: 0': 'Тип отзыва','Unnamed: 1' : 'Текст отзыва', 'Unnamed: 2': 'Типы проблем'}, inplace=True)
df3.rename(columns={'Unnamed: 0': 'Тип отзыва','Unnamed: 1' : 'Текст отзыва', 'Unnamed: 2': 'Типы проблем'}, inplace=True)
df_combined = pd.concat([df1, df2, df3], ignore_index=True)
df_combined.rename(columns={'Типы проблем': 'Спам'}, inplace=True)
df_combined['Спам'] = df_combined['Спам'].apply(lambda x: 1 if x == 'Спам, удалить' else 0)
df_combined['Тип отзыва'] = df_combined['Тип отзыва'].apply(lambda x: 1 if x == 'позитивный' else 0)

Признак `Спам`: 1 - спам, 0 - не спам

Признак `Тип отзыва`: 1 - позитивный, 0 - негативный

In [5]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

morph = MorphAnalyzer()

def preprocess_text(text):
    text = re.sub(r"[^\w\s]", "", text.lower())
    text = re.sub(r"\s+", " ", text).strip()
    words = text.split()
    lemmatized_text = " ".join(morph.parse(word)[0].normal_form for word in words)
    return lemmatized_text

def get_bert_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

df_combined['Текст отзыва'] = df_combined['Текст отзыва'].apply(preprocess_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
df_combined.sample(15)

,Тип отзыва,Текст отзыва,Спам
20745,1,всё отлично вкусно и спасибо за блюдо за балл,0
18461,1,самый вкусный ролл в ярославль идеально сварит...,0
8800,1,вкусно быстро спасибо,0
31674,1,как всегда сочно вкусно сытно,0
30446,0,пицца без подложка на день коробка лежать крем...,0
26539,1,как всегда всё замечательный,0
22857,1,ролл с креветка особенно классный и уда понрав...,0
34123,0,принести 42 штука а должный быть 48 то есть из...,0
24459,1,брать в это ресторан ролл ролл понравиться быт...,0
1122,0,суп быть уже почти холодный и не вкусный рис к...,0


## Шаг 1. Классификация спам-отзывов

Определяем, спам-отзыв это или нет. Также ставим порог (0.7) для сомнительных отзывов, чтобы те сохранялись в отдельный документ и переходили на дополнительную модерацию с аннотацией

In [ ]:
import torch

checkpoint = torch.load("spam_clsf.pth")

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_config(checkpoint["config"])
model.load_state_dict(checkpoint["model_state_dict"])

tokenizer = checkpoint["tokenizer"]

print("Модель и токенизатор успешно загружены")

In [ ]:
# Лемматизатор
morph = MorphAnalyzer()

def preprocess_text(text):
    # Убираем спецсимволы, переводим в нижний регистр
    text = re.sub(r"[^\w\s]", "", text.lower())
    # Исправляем пробелы и сокращения
    text = re.sub(r"\s+", " ", text).strip()
    # Лемматизация
    words = text.split()
    lemmatized_text = " ".join(morph.parse(word)[0].normal_form for word in words)
    return lemmatized_text

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

def get_bert_embeddings(texts, tokenizer, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    model.eval()

    embeddings = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [CLS] токен
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [ ]:
data = pd.read_csv('com2.csv').head(10)
data["Текст отзыва"] = data["Текст отзыва"].apply(preprocess_text)
texts = data["Текст отзыва"].tolist()
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

data["Предсказание (Спам)"] = predictions.numpy()

print(data)

## Шаг 2. Анализ тональности

Определяем тип отзыва - негативный / позитивный. Для негативных выставляем флаг `Спам, удалить`.

In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", model="roberta-base")

def analyze_sentiment(review):
    result = sentiment_analyzer(review)
    return result[0]['label']  # 'POSITIVE' or 'NEGATIVE'

## Шаг 3. Анализ контекста

Если отзыв имеет тип "Негативный", то выявляем проблемы. А если отзыв "Позитивный" - причины хорошего отзыва


### zero-shot подход

In [77]:
model = SentenceTransformer('all-mpnet-base-v2')

problem_list = [
    "Доставка очень долгая",
    "Еда была холодной",
    "Курьер вел себя грубо",
    "Привезли не тот заказ",
    "Цена слишком высокая",
]

problem_embeddings = model.encode(problem_list, convert_to_tensor=True)
problem_embeddings = normalize(problem_embeddings.cpu().numpy(), axis=1)

def match_problems_cosine(review_text):
    review_embedding = model.encode(review_text, convert_to_tensor=True)
    review_embedding = normalize(review_embedding.cpu().numpy().reshape(1, -1))

    similarities = cosine_similarity(review_embedding, problem_embeddings)[0]
    problem_scores = [(problem, similarity) for problem, similarity in zip(problem_list, similarities)]
    sorted_problems = sorted(problem_scores, key=lambda x: x[1], reverse=True)

    threshold = 0.5
    return [problem for problem in sorted_problems if problem[1] > threshold]

# Пример отзыва
review = "доставку я не получила. Два часа ждала"
matched_problems = match_problems_cosine(review)

print("Две наиболее вероятные проблемы:")
for problem, similarity in matched_problems[:2]:
    print(f"- {problem}: {similarity:.2f}")

Две наиболее вероятные проблемы:
- Доставка очень долгая: 0.75
- Еда была холодной: 0.64


### Выявление причины позитивного отзыва

In [34]:
generator = pipeline("text2text-generation", model="t5-base")

def positive_reason(review):
    input_text = f"Why is this review positive? {review}"
    result = generator(input_text, max_length=50)
    return result[0]['generated_text']

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## Шаг 4. Выделение эмоций

Оптимально использовать 6 основных эмоций: радость, грусть, гнев, страх, удивление, отвращение. Эти эмоции универсальны и покрывают большинство сценариев.

In [78]:
emoberta = pipeline("text-classification", model="tae898/emoberta-base")

def detect_emotion(review):
    emotions = emotion_analyzer(review)
    return max(emotions, key=lambda x: x['score'])['label']  # e.g., 'anger'

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/407 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

## Шаг 5. Генерация рекомендаций

Используем GPT для построения рекомендаций. Рекомендации можно отправлять ресторанам - партнерам для лучшей коммуникации.


In [79]:
def generate_recommendation(problem):
    prompt = f"Provide a recommendation for the problem: {problem}"
    result = generator(prompt, max_length=50)
    return result[0]['generated_text']

# EDA

In [7]:
df_combined.head()

,Тип отзыва,Текст отзыва,Спам
0,0,доставка быстрый начинка мало,0
1,0,не доварёный рис,0
2,0,привезти в установленный время и на это плюс з...,0
3,0,ужасно ужасный клиентоориентированность не поз...,0
4,0,1 задержка от изначально ориентировочный срок ...,0


In [27]:
df_pie = df_combined.copy()
df_pie['Тип отзыва'] = df_combined['Тип отзыва'].map({0: 'Негатив', 1: 'Позитив'})

fig = px.pie(df_pie,
             names='Тип отзыва',
             title='Распределение отзывов по типам',
             hole=0.3)

fig.update_traces(textinfo='percent+label')
fig.show()

In [29]:
df_pie['Спам'] = df_combined['Спам'].map({0: 'Не спам', 1: 'Спам'})

fig = px.pie(df_pie,
             names='Спам',
             title='Распределение спама',
             hole=0.3)

fig.update_traces(textinfo='percent+label')
fig.show()